Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 3.52 ms, sys: 520 µs, total: 4.04 ms
Wall time: 11.7 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.1.0.87/6484/1 Dashboard: http://10.1.0.87:8787/status,Cluster Workers: 1 Cores: 6 Memory: 7.29 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 212 ms, sys: 24.1 ms, total: 236 ms
Wall time: 2.74 s


[1.6525209751131515,
 1.8675982852345254,
 1.8844831975414822,
 1.1098549048564532,
 1.176303930764031,
 1.3173251352769664,
 1.7926126820084616,
 1.2066553747199227,
 1.8517633989938802,
 1.3728552982877376,
 1.772636079535972,
 1.9791592913995801,
 1.5843869039750305,
 1.2943445805492764,
 1.7871409038008215,
 1.916656056282126,
 1.9964531365466942,
 1.8240945816006913,
 1.637007287286247,
 1.4954310462078975,
 1.9419568054330836,
 1.8105582649984138,
 1.1093537355238554,
 1.5295387004358676,
 1.5496712582361711,
 1.5582413956897923,
 1.9951618467547023,
 1.6468077816495597,
 1.6705981578438558,
 1.1059426315902,
 1.4515958633736308,
 1.1771290361338471,
 1.1376525433703866,
 1.9978800352190933,
 1.9540341210796939,
 1.9873347108443467,
 1.6082872975291846,
 1.2067657967016885,
 1.317602022560824,
 1.4490501490162524,
 1.7103751126858437,
 1.5585286386842145,
 1.2903562413794332,
 1.8390272928295854,
 1.5274405957274402,
 1.5376525756301,
 1.6354802318744066,
 1.2543344333995865,
 1.

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.9579202935935982,
 1.6918611917310682,
 1.4731519391588477,
 1.5166394657690463,
 1.4980439309400992,
 1.920158299577218,
 1.577019811706089,
 1.8284093879280676,
 1.9415201342424238,
 1.986438617202496,
 1.3326005991267231,
 1.5886781262887706,
 1.7152231006929264,
 1.848894526360174,
 1.0649320459012181,
 1.0976101340880515,
 1.9579798981776158,
 1.5730328564643221,
 1.6532213645170273,
 1.4216803887545868,
 1.236917533119278,
 1.6001045729394487,
 1.2792894936256163,
 1.7222254374012933,
 1.8215825553293459,
 1.1146880047133145,
 1.168292337684063,
 1.9709369293960122,
 1.948894070432009,
 1.0103782443882092,
 1.4837889045698662,
 1.807463619456891,
 1.8104227740019916,
 1.8251077178470794,
 1.4238093233789826,
 1.1013504668769105,
 1.8112736282227293,
 1.4167801361077084,
 1.4206650630715303,
 1.0950333449085314,
 1.7396949091516667,
 1.428758039842041,
 1.8960183399012056,
 1.2097903449409242]

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 199 ms, sys: 18.9 ms, total: 217 ms
Wall time: 2.58 s
